# What is Trump yelling about?

## Setup

In [115]:
import twitter

In [100]:
# Define your api_key/secret and access_token_key/secret here

api = twitter.Api(consumer_key=api_key,
                      consumer_secret=api_secret,
                      access_token_key=access_token_key,
                      access_token_secret=access_token_secret)

In [116]:
api.VerifyCredentials()

User(ID=1961548171, ScreenName=kmarwahaha)

### Scrape last ~3200 tweets

In [117]:
def get_statuses(prev_statuses = None, screen_name = "realDonaldTrump", count=200):
    max_id = prev_statuses[-1].id-1 if prev_statuses else None
    return api.GetUserTimeline(screen_name=screen_name, max_id=max_id, count=count)

In [118]:
n = 16
temp = get_statuses()
all_statuses = temp
for _ in range(n):
    temp = get_statuses(temp)
    all_statuses += temp

In [119]:
# Earliest tweet scraped
all_statuses[-1]

Status(ID=784101360736370688, ScreenName=realDonaldTrump, Created=Thu Oct 06 18:41:46 +0000 2016, Text=u"RT @DonaldJTrumpJr: Great group at our Victory Office in Columbus, Ohio. I'm incredibly grateful to have so many\u2026 https://t.co/rLJWCAGRlW")

### Parse date

In [120]:
#https://stackoverflow.com/questions/3682748/converting-unix-timestamp-string-to-readable-date-in-python
import datetime
def readable_date(unix_time):
    return datetime.datetime.fromtimestamp(int(unix_time)).strftime('%Y.%m.%d')

In [121]:
text_and_date = [(status.text, readable_date(status.created_at_in_seconds)) for status in all_statuses]

In [122]:
# Most recent tweet
text_and_date[0]

(u'As a candidate, I promised we would pass a massive TAX CUT for the everyday working American families who are the b\u2026 https://t.co/sTAoTR50EF',
 '2017.12.17')

## Analyze

In [128]:
def trim_results(text_and_date, filterx):
    return filter(
        lambda y: len(y[1]) > 0,
        [(date, 
            " ".join(filter(filterx, 
                            map(lambda x: x.replace('-', ' ').replace(u'\u2026', '').strip(), 
                                text.strip().split())
            ))
        ) for (text, date) in text_and_date]
    )

### Approach 1: Find words that are uppercase, capitalized, hashtag'd, or end in "!"

In [130]:
filter1 = lambda x: x != u'&amp;' and (len(x) > 0 and x[0].upper() == x[0] 
                   or x.upper() == x or u'#' == x[0] or u'!' == x[-1])

In [136]:
trimmed1 = trim_results(text_and_date, filter1)

In [137]:
trimmed1[:20]

[('2017.12.17', u'As I TAX CUT American'),
 ('2017.12.16', u'WEEKLY ADDRESS\U0001f1fa\U0001f1f8'),
 ('2017.12.16', u'TAX CUTS American U.S.'),
 ('2017.12.16', u'Congratulations Corey Lewandowski David Bossie,'),
 ('2017.12.15', u'Great Kevin, you!'),
 ('2017.12.15', u'RT @RepKristiNoem: A 30+'),
 ('2017.12.15',
  u'RT @SpeakerRyan: For Tax Cuts Jobs Act: \u2714lowers \u2714nearly'),
 ('2017.12.15', u'RT @PressSec: .@POTUS +'),
 ('2017.12.15', u'DOW, S&amp;P 500 NASDAQ highs! #MAGA'),
 ('2017.12.15',
  u'Today, Marine Helicopter Squadron One (HMX 1) Marine Corps Air Facility'),
 ('2017.12.15', u'You \u2013 THE MEN AND WOMEN IN BLUE. Thank'),
 ('2017.12.15', u'To National Academy Quantico, CONGRATULATIONS!'),
 ('2017.12.15', u'It THANK YOU!'),
 ('2017.12.14',
  u'RT @GOPChairwoman: .@realDonaldTrump Paycheck President. Learn'),
 ('2017.12.14',
  u'\u201cManufacturing Optimism Rose Another All Time High Latest @ShopFloorNAM Outlook Survey\u201d'),
 ('2017.12.14', u'In 1960, 20,000 Code Fe

Hmm, this isn't that insightful. Let's see what he's really yelling about.

### Approach 2: Just look for YELLING

In [77]:
#https://stackoverflow.com/questions/1265665/python-check-if-a-string-represents-an-int-without-using-try-except
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
def IsASCII(u):
    try: 
        u.encode()
        return True
    except UnicodeEncodeError:
        return False

In [78]:
filter2 = lambda x: x.upper() == x and not RepresentsInt(x) and u'-' != x and x != u'RT' and len(x) > 3 and IsASCII(x)

In [144]:
trimmed2 = trim_results(text_and_date, filter2)

In [146]:
trimmed2[:20]

[('2017.12.16', u'WEEKLY'),
 ('2017.12.16', u'CUTS U.S.'),
 ('2017.12.15', u'.@POTUS'),
 ('2017.12.15', u'DOW, NASDAQ #MAGA'),
 ('2017.12.15', u'(HMX 1)'),
 ('2017.12.15', u'WOMEN BLUE.'),
 ('2017.12.15', u'CONGRATULATIONS!'),
 ('2017.12.15', u'THANK YOU!'),
 ('2017.12.14', u'1960, 20,000 185,000'),
 ('2017.12.14', u'TAPE!'),
 ('2017.12.13', u'GREAT'),
 ('2017.12.09', u'TROPHY!'),
 ('2017.12.09', u'FAKE'),
 ('2017.12.09', u'U.S.'),
 ('2017.12.09', u'CNN, MOST TRUSTED NAME NEWS.'),
 ('2017.12.09', u'GREAT EVENING'),
 ('2017.12.09', u'NATION'),
 ('2017.12.08', u'LIVE #MAGA'),
 ('2017.12.08', u'#MAGA'),
 ('2017.12.08', u'4.1%. 2.6%,')]

That's a bit better.

### What's his yelling vocabulary like?

In [165]:
from collections import Counter

In [166]:
trump_yelling = Counter([x[1] for x in trimmed])

In [169]:
[i for i in trump_yelling.most_common() if i[1] > 2]

[(u'U.S.', 78),
 (u'MAKE AMERICA GREAT AGAIN!', 34),
 (u'#MAGA', 32),
 (u'D.C.', 18),
 (u'@FLOTUS', 12),
 (u'GREAT', 11),
 (u'FAKE NEWS', 11),
 (u'#ICYMI', 9),
 (u'JOBS, JOBS, JOBS!', 9),
 (u'@POTUS', 9),
 (u'@CNN', 9),
 (u'#MAGA!', 8),
 (u'FAKE NEWS!', 7),
 (u'LIVE', 7),
 (u'ISIS', 7),
 (u'@FLOTUS:', 6),
 (u'FEMA', 6),
 (u'#UNGA', 6),
 (u'U.S.A.', 5),
 (u'2017,', 5),
 (u'.@POTUS', 5),
 (u'#ICYMI:', 5),
 (u'WIN!', 5),
 (u'NEVER', 5),
 (u'THANK YOU!', 4),
 (u'9:00 P.M.', 4),
 (u'VERY', 4),
 (u'NOTHING', 4),
 (u'@USGA', 4),
 (u'A.G.', 4),
 (u'VIDEO:', 4),
 (u'33,000', 4),
 (u'116%', 4),
 (u'MAKE AMERICA GREAT AGAIN', 4),
 (u'VOTE', 4),
 (u'W.H.', 4),
 (u'V.A.', 3),
 (u'HAPPY BIRTHDAY', 3),
 (u'7:00 P.M.', 3),
 (u'#USA', 3),
 (u'@FEMA', 3),
 (u'FAKE', 3),
 (u'USA!', 3),
 (u'11:00 A.M.', 3),
 (u'100%', 3),
 (u'.@FLOTUS', 3),
 (u'@ABC', 3),
 (u'9:00 A.M.', 3),
 (u'MAKE AMERICA SAFE GREAT AGAIN!', 3),
 (u'A.C.', 3),
 (u'WEEKLY', 3),
 (u'6:00 A.M.', 3),
 (u'ISIS,', 3),
 (u'ZERO', 3),
 (u'U.S.

Makes sense. His whole YELLING vocab is below.

In [173]:
sorted(trump_yelling.keys())

[u'"46%',
 u'"77% U.S. (WT) DANGEROUS!',
 u'"@AZTRUMPTRAIN:',
 u'"@POTUS',
 u'"@THEREALMOGUL:',
 u'"DEPLORABLES."',
 u'"DRAIN SWAMP" DTS.',
 u'"MESS," CORRUPT!',
 u'"MOVEMENT" D.C.',
 u'"NO"',
 u'"REASONS VOTE DEMOCRATS"',
 u'"TRUMP TRAIN"!!"',
 u'"YOU NEED BOTH PUBLIC PRIVATE POSITION"',
 u'#AHCA.',
 u'#APEC',
 u'#APEC2017',
 u'#APEC2017 @FLOTUS',
 u'#ASEAN50 FAIR TRADE DEALS,',
 u'#DOW 16%. #NASDAQ 19.5%.',
 u'#DTS',
 u'#FAKE NEWS!',
 u'#FEMA',
 u'#FNN',
 u'#GA06',
 u'#GES2017',
 u'#GES2017.',
 u'#ICYMI',
 u'#ICYMI EO1: EO2:',
 u'#ICYMI NARRATIVE FIGHT." #MAGA',
 u'#ICYMI OHIO RALLY!',
 u'#ICYMI,',
 u'#ICYMI:',
 u'#ICYMI: 11/8.',
 u'#ISIS',
 u'#LESM',
 u'#LIUB.',
 u'#MAGA',
 u'#MAGA WINNING,',
 u'#MAGA!',
 u'#MAGA! #ICYMI',
 u'#MAGA.',
 u'#MS13 #MS13',
 u'#MSM LOVED',
 u'#NABTU2017',
 u'#NABTU2017.',
 u'#NATO',
 u'#PEOTUS #VPE',
 u'#PEOTUS #VPEOTUS #MAGA',
 u'#POTUS',
 u'#POTUS7',
 u'#SCOTUS',
 u'#TARC',
 u'#UNGA',
 u'#USA',
 u'#USAF B 1B',
 u'#USMC242',
 u'#USVI.',
 u'#USVI. @FEMA',

### Saving and cleanup

In [180]:
%store all_statuses > all_statuses.txt
%store text_and_date > text_and_date.txt
%store trimmed1 > trimmed1.txt
%store trimmed2 > trimmed2.txt
%store trump_yelling > trump_yelling.txt

Writing 'all_statuses' (list) to file 'all_statuses.txt'.
Writing 'text_and_date' (list) to file 'text_and_date.txt'.
Writing 'trimmed1' (list) to file 'trimmed1.txt'.
Writing 'trimmed2' (list) to file 'trimmed2.txt'.
Writing 'trump_yelling' (Counter) to file 'trump_yelling.txt'.


In [181]:
%store -r